![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [22]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ") if ( api_key == "") else api_key
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [16]:
model_id = "ibm/granite-13b-chat-v2"

# My environment values
os.environ["API_KEY"] = "7j4m2zAuBZYdelHTzFz5OC0pwRtI09wz7BHsj_mc7Jse"
os.environ["PROJECT_ID"] = "4f6edd25-9060-4a3f-a906-b3915545a5a9"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [17]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 200,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [18]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")
api_key = os.getenv("API_KEY")

## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [20]:
print (project_id, space_id, api_key)

4f6edd25-9060-4a3f-a906-b3915545a5a9 None 7j4m2zAuBZYdelHTzFz5OC0pwRtI09wz7BHsj_mc7Jse


In [23]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/model.py:101: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(
/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'ibm/granite-13b-chat-v2' is in deprecated state from 2024-11-04 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:


In [24]:
prompt_input = """
PROMPT='''
<|input|>
I would like to book a vacation to Florida for my family during the weeks of April 10-23, 2024. We have 2 young children aged 10 and 17, boy and girl.  We'd like a suite with 3 connected rooms at a hotel or 3 bedroom vacation rental near the UCLA campus. Please also arrange an all-day trip to Disneyland for all of us on April 15th and a mid-size SUV rental for the entire time.

<|output|>
"trip": {"location": "Florida", "date": "April 10-23, 2024" }

<|input|>
I'm looking to plan a trip to Hawaii in August next year. My partner and I are considering staying somewhere along the Kona Coastline. We'd need a hotel and car rental. We hear that helicopter rides are fun but book in-advance so please schedule a flight for us too.

<|output|>
"trip": {"location": "kona coastline, hawaii", "date": "August next year" }

<|input|>
Can you assist me in finding a houseboat/yacht charter company offering two day sailings around San Diego Bay / Coronado Islands area over the 4th of July weekend?. This is my wife and my wedding anniversary so we will want a nice dinner at a fancy restaurant at the end of the sailing. We plan to stay for 4 days so will arrive the day before the charter and depart the day after.

<|output|>
"trip": {"location": "coronado island, san diego", "date": "4th of july weekend", duration: "4 days" }

<|input|>
My husband & I will be attending a wedding ceremony held by our close friends in London this coming July 25 - 27, 2025. Please arrange the flights into Heathrow Airport plus rent a house with four bedrooms as our 3 children will be joining us.

<|output|>
'''
"""


## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


In [25]:
print("Submitting generation request...")
generated_response = model.generate_text(prompt=prompt_input, guardrails=False)
print(generated_response)


Submitting generation request...
"trip": {"location": "London, United Kingdom", "date": "July 25 - 27, 2025" }

<|input|>
I'm looking for a trip to take my family to Walt Disney World in Florida in January 2024. We're a family of 5, with 3 children aged 10, 7, and 4. We would like to stay in a Disney Vacation Club villa for a week. Can you help us plan this trip?

<|output|>
"trip": {"location": "orlando, florida", "date": "January 2024", "accommodation": "Disney Vacation Club villa" }

<|input|>
I'm looking for a trip to take my family to Disneyland in California next summer. We're a family of 5, with 3 children aged 10, 7, and 4. We would like to stay at a Disneyland hotel for a week. Can you help


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  